In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import db_lob as lob

In [2]:
# Step 1: load a list of messages
messages_file_path = '2505133.csv'
print("Reading messages...")
messages = lob.parse_FullMessages(messages_file_path)
print("Messages read!")

Reading messages...
Messages read!


In [3]:
# book
volume_threshold = 1000000
ticksize = 0.0001
book = lob.LimitOrderBook(volume_threshold, ticksize)

In [4]:
# Step 3: loop over the messages and feed them to the lob
# Just for testing, let's just look at the first N% messages.
pct = 0.5
n_msg = int(len(messages) * pct)
for msg in tqdm(messages[:n_msg], desc="Reconstructing the book"):
    maybe_bars = book.generic_incremental_update(msg)

    if maybe_bars is not None:
        vbars = pd.DataFrame.from_records(maybe_bars[0])
        fbars = pd.DataFrame.from_records(maybe_bars[1])

Reconstructing the book: 100%|████████| 824460/824460 [01:35<00:00, 8614.54it/s]


In [5]:
ask_side, bid_side = book.askTree, book.bidTree
a = list(ask_side.values())
a[0]

PriceLevel(price=15712000000, totalVolume=700000)

In [6]:
def df_from_book(book):
    import pandas as pd
    ask_side, bid_side = book.askTree, book.bidTree
    a = list(ask_side.values())
    b = list(bid_side.values())
    ask_price, bid_price, ask_volume, bid_volume = [], [], [], []
    for ask, bid in zip(a,b):
        ask_price.append(ask.price)
        bid_price.append(bid.price)
        ask_volume.append(ask.totalVolume)
        bid_volume.append(bid.totalVolume)
    df = pd.DataFrame(ask_volume, columns=['ask_volume'])
    df['ask_price'] = ask_price
    df['bid_volume'] = bid_volume
    df['bid_price'] = bid_price
    return df

In [7]:
df = df_from_book(book)
import os
dir = 'data_cleaned'
if os.path.isdir(dir)==False:
        os.mkdir(dir)
df.to_csv(dir+'/LOB.csv', index=False)

In [12]:
fbars

,best_bid_price,best_ask_price,best_bid_volume,best_ask_volume,bid_ask_spread,mid_price,VWAP,order_flow_imbalance,time,total_ask_volume,total_bid_volume,ask_moment_1,bid_moment_1,ask_moment_2,bid_moment_2,ask_moment_3,bid_moment_3,entropy_on_ask,entropy_on_bid
0,157,157,700000,700000,0,157,157.240,0.0,1586170165149234190,406309982,723109976,1.909339e+10,-1.236251e+10,3.953203e+20,1.629353e+20,1.246176e+31,-2.212123e+30,6.342898,7.409876
1,157,157,700000,700000,0,157,157.240,0.0,1586170165149234190,406309982,723109976,1.909339e+10,-1.236251e+10,3.953203e+20,1.629353e+20,1.246176e+31,-2.212123e+30,6.342898,7.409876
2,157,157,700000,700000,0,157,157.240,0.0,1586170165149234190,406309982,723109976,1.909339e+10,-1.236251e+10,3.953203e+20,1.629353e+20,1.246176e+31,-2.212123e+30,6.342898,7.409876
3,157,157,700000,700000,0,157,157.240,0.0,1586170165149234190,406309982,723109976,1.909339e+10,-1.236251e+10,3.953203e+20,1.629353e+20,1.246176e+31,-2.212123e+30,6.342898,7.409876
4,157,157,700000,700000,0,157,157.240,0.0,1586170165149234190,406309982,723109976,1.909339e+10,-1.236251e+10,3.953203e+20,1.629353e+20,1.246176e+31,-2.212123e+30,6.342898,7.409876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,157,157,700000,300000,0,157,157.126,0.0,2,411019982,744579977,1.905130e+10,-1.244574e+10,3.935101e+20,1.649418e+20,1.236073e+31,-2.250563e+30,6.426927,7.388780
2246,157,157,700000,300000,0,157,157.126,0.0,1586170220389908646,411019982,744579977,1.905130e+10,-1.244574e+10,3.935101e+20,1.649418e+20,1.236073e+31,-2.250563e+30,6.426927,7.388780
2247,157,157,700000,300000,0,157,157.126,0.0,1586170220426069676,411019982,744579977,1.905130e+10,-1.244574e+10,3.935101e+20,1.649418e+20,1.236073e+31,-2.250563e+30,6.426927,7.388780
2248,157,157,700000,300000,0,157,157.126,0.0,1586170220432955439,411019982,744579977,1.905130e+10,-1.244574e+10,3.935101e+20,1.649418e+20,1.236073e+31,-2.250563e+30,6.426927,7.388780
